# BM25 Integration File

In [1]:
K_Passages = 5

PATH_INTEGRATION_CLASSIFIER = '/mnt/ciaamS03/mnt/Files/integration/bm25/'

INDEX_KNOWLEDGE_BASE = ["biomedical_100_pt_true"]


## Starting Elastic Search server

In [2]:
#! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
#! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
#! chown -R daemon:daemon elasticsearch-7.9.2
##! chmod 755 -R elasticsearch-7.9.2/bin/elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elastic_docs/elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

## Loading document store 

In [3]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="")

10/22/2021 21:58:38 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
10/22/2021 21:58:39 - INFO - faiss.loader -   Loading faiss with AVX2 support.
10/22/2021 21:58:39 - INFO - faiss.loader -   Loading faiss.
10/22/2021 21:58:39 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.046s]
10/22/2021 21:58:39 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.008s]
10/22/2021 21:58:39 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.004s]
10/22/2021 21:58:39 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.018s]
10/22/2021 21:58:39 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


## Creating Retriever

In [4]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [5]:
from haystack.pipeline import Pipeline

extractive_pipeline = Pipeline()
extractive_pipeline.add_node(component=retriever, name="ESRetriever1", inputs=["Query"])

## Function to get main documents

In [6]:
def get_passages(question):
    
    res = extractive_pipeline.run(query=question, top_k_retriever=K_Passages, index = INDEX_KNOWLEDGE_BASE)
    
    documents = ""
    for i in range (K_Passages):
        if i < len(res["documents"]):
            document_dict = ast.literal_eval(str(res["documents"][i]))
            documents += document_dict["text"] + " "
    question_with_passages = "question: " + question +" [SEP] context: "+documents
     
    return(question_with_passages)

## Main loop that receives a question and sends the passages back

In [ ]:
#aguarda pergunta e responde a resposta (inferência)

import csv
import time
import os
import ast


path = "/mnt/ciaamS03/mnt/Files/integration/bm25/"
print(f"Listening path: {path}")

while 1:#loop infinito tem que parar manualmente
    time.sleep(0.5)#em segundos, delay para não consumir todo o processador da máquina.
    
    ready = os.path.exists(path + "in/ready.txt")
    question = os.path.exists(path + "in/question.csv")
    #print(f"Ready = {ready}\nQuestion = {question}")
        
    if ready == True and question == True:
        print("\nNew request arrive:")
                
        csv_out = open(path + "out/queries.csv", "w", encoding='utf8')
        ready_out = open(path + "out/ready.txt", "w", encoding='utf8')
        ready_out.write("Done")#qualquer conteúdo       
       
        
        with open(path + "in/question.csv", newline = '', encoding='utf8') as csv_in:
            csv_in_reader = csv.reader(csv_in, delimiter=';')
            for csv_i in csv_in_reader:
                print(csv_i[0])
                code = get_passages(csv_i[0])               
                print(code)
                csv_out.write(f"{csv_i[0]};{code}\n")
                break #só o primeiro               
        csv_out.close()   
        ready_out.close()
        os.remove(path + "in/ready.txt") #Deletado indica que terminou o processamento
        os.remove(path + "in/question.csv")#Deletado indica que terminou o processamento

Listening path: /mnt/ciaamS03/mnt/Files/integration/bm25/


10/22/2021 23:19:07 - INFO - elasticsearch -   POST http://localhost:9200/biomedical_100_pt_true/_search [status:200 request:0.100s]



New request arrive:
Quais são os três procedimentos mais caros
question: Quais são os três procedimentos mais caros [SEP] context: Em geral, os procedimentos de cirurgia de bypass gástrico são mais caros, custando no intervalo de US $ 18.000 a US $ 35.000, enquanto os procedimentos de bandas gástricos são executados de US $ 17.000 para US $ 30.000. O custo da cirurgia inclui as seguintes taxas: Custo da cirurgia bariátrica. O custo da cirurgia de perda de peso bariátrica é tipicamente cerca de US $ 25.000. Em geral, os procedimentos de cirurgia de bypass gástrico são mais caros, custando no intervalo de US $ 18.000 a US $ 35.000, enquanto os procedimentos de faixas gástricas são executados de US $ 17.000 a US $ 30.000. O custo da cirurgia inclui as seguintes taxas: 1 Taxas hospitalares. O custo da cirurgia de perda de peso bariátrica é tipicamente cerca de US $ 25.000. Mas não conseguir que o dente fixo pudesse custar ainda mais.xtrações (tendo um dente puxado) geralmente custam entre

10/23/2021 11:14:42 - INFO - elasticsearch -   POST http://localhost:9200/biomedical_100_pt_true/_search [status:200 request:0.037s]



New request arrive:
quanto tempo depois da gripe eu sou infecciosoquanto tempo depois da gripe eu sou infeccioso
question: quanto tempo depois da gripe eu sou infecciosoquanto tempo depois da gripe eu sou infeccioso [SEP] context: Resposta: Então, se você estiver perto de alguém que tenha a gripe, e no dia seguinte você se sente muito bem, você não está fora da floresta ainda. Você pode realmente desenvolver sintomas de até quatro a cinco dias após ser exposto a alguém que tenha a gripe. Próximo: Por quanto tempo eu sou contagiante depois de ser infectado pela gripe? Toque Confiança Votos 202. Vai doer tê-lo puxado quando é abscesso, mas se sentirá melhor muito logo depois. Eu tinha um puxado enquanto estava abscesso e era muito doloroso, mas eu sou muito grato que eu fiz isso. Eu não poderia ter ficado a dor do abscesso por muito mais tempo. Cura após uma extração de dentes pode levar até uma semana. Pmsing cedo? BDL13. Eu sou devido ao meu período em cerca de 2 semanas, mas nos últi

10/23/2021 11:36:14 - INFO - elasticsearch -   POST http://localhost:9200/biomedical_100_pt_true/_search [status:200 request:0.015s]



New request arrive:
quanto tempo depois da gripe eu sou infeccioso
question: quanto tempo depois da gripe eu sou infeccioso [SEP] context: Resposta: Então, se você estiver perto de alguém que tenha a gripe, e no dia seguinte você se sente muito bem, você não está fora da floresta ainda. Você pode realmente desenvolver sintomas de até quatro a cinco dias após ser exposto a alguém que tenha a gripe. Próximo: Por quanto tempo eu sou contagiante depois de ser infectado pela gripe? Toque Confiança Votos 202. Vai doer tê-lo puxado quando é abscesso, mas se sentirá melhor muito logo depois. Eu tinha um puxado enquanto estava abscesso e era muito doloroso, mas eu sou muito grato que eu fiz isso. Eu não poderia ter ficado a dor do abscesso por muito mais tempo. Cura após uma extração de dentes pode levar até uma semana. Quanto tempo dura uma pessoa com a gripe contagiosa? Você pode passar a gripe para os outros de 1 dia antes de ter sintomas até 5-7 dias depois de ficar doente. Algumas pessoas

10/23/2021 11:40:52 - INFO - elasticsearch -   POST http://localhost:9200/biomedical_100_pt_true/_search [status:200 request:0.031s]



New request arrive:
O que causa dor nas costas
question: O que causa dor nas costas [SEP] context: Sintomas de dor nas costas. Sintomas que geralmente ocorrem com a dor nas costas incluem rigidez traseira, espasmos musculares e ternura sobre os músculos das costas. Sintomas que indicam uma causa séria para a dor nas costas incluem dor nas costas intensa que atira a perna, dormência, fraqueza da perna, dormência da virilha e dificuldade em urinar. Em casos de dor nas costas crônicas, pode pensar: "As úlceras causam dor nas costas?" Uma resposta simples a essa pergunta ansiosa é "sim", uma úlcera péptica é uma provável causa de dor nas costas. Dor nas costas da moeda aguda. A dor nas costas da moeda aguda é geralmente o resultado de uma lesão muscular súbita ou superexerção. Em praticamente todos os casos, a dor é precedida por algum evento que realmente causa trauma ou é percebido para causar ferimentos. A dor nas costas pode vir de repente e durar menos de seis semanas (aguda), que po